In [1]:
!pip install transformers sentencepiece

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move model to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print("Model loaded on", device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Model loaded on cuda


In [3]:
from google.colab import files
uploaded = files.upload()

Saving ne_si_dev.jsonl to ne_si_dev.jsonl


In [4]:
import json

ne_sents = []
si_refs = []

with open("ne_si_dev.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        ne_sents.append(data["src"])
        si_refs.append(data["tgt"])

print(f"Loaded {len(ne_sents)} sentence pairs")

Loaded 997 sentence pairs


In [5]:
def translate_batch(inputs, src_lang="npi_Deva", tgt_lang="sin_Sinh"):
    print("⚙️  Translating batch of", len(inputs))

    tokenizer.src_lang = src_lang
    encoded = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True, max_length=512)
    encoded = {k: v.to("cuda") for k, v in encoded.items()}  # Force GPU here

    forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=forced_bos_token_id, max_length=512)

    decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return decoded

In [6]:
from tqdm import tqdm

batch_size = 8
predicted_si = []

for i in tqdm(range(0, len(ne_sents), batch_size), desc="Translating"):
    batch = ne_sents[i:i+batch_size]
    preds = translate_batch(batch)
    predicted_si.extend(preds)

# Save predictions
with open("direct_predicted_si.txt", "w", encoding="utf-8") as f:
    for line in predicted_si:
        f.write(line.strip() + "\n")

print("Translations saved to 'direct_predicted_si.txt'")

Translating:   0%|          | 0/125 [00:00<?, ?it/s]

⚙️  Translating batch of 8


Translating:   1%|          | 1/125 [00:12<26:28, 12.81s/it]

⚙️  Translating batch of 8


Translating:   2%|▏         | 2/125 [00:14<13:20,  6.51s/it]

...

⚙️  Translating batch of 8


Translating:  99%|█████████▉| 124/125 [04:46<00:01,  1.88s/it]

⚙️  Translating batch of 5


Translating: 100%|██████████| 125/125 [04:48<00:00,  2.31s/it]

Translations saved to 'direct_predicted_si.txt'


In [1]:
!rm -rf bleurt
!pip install git+https://github.com/google-research/bleurt.git
!pip install tensorflow  # BLEURT requires TensorFlow

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-vbc44u2b
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-vbc44u2b
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


In [2]:
!wget https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
!unzip bleurt-base-128.zip

--2025-04-23 17:34:00--  https://storage.googleapis.com/bleurt-oss/bleurt-base-128.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 192.178.163.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|192.178.163.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405489453 (387M) [application/zip]
Saving to: ‘bleurt-base-128.zip.1’

bleurt-base-128.zip 100%[===================>] 386.70M   211MB/s    in 1.8s    

2025-04-23 17:34:02 (211 MB/s) - ‘bleurt-base-128.zip.1’ saved [405489453/405489453]

Archive:  bleurt-base-128.zip
   creating: bleurt-base-128/
  inflating: bleurt-base-128/vocab.txt  
  inflating: bleurt-base-128/bert_config.json  
   creating: bleurt-base-128/variables/
  inflating: bleurt-base-128/variables/variables.index  
  inflating: bleurt-base-128/variables/variables.data-00000-of-00001  
  inflating: bleurt-base-128/bleurt_config.json  
  inflating: bleurt-base-128/saved_model.pb

In [11]:
with open("/content/ne_si_dev.jsonl", "r", encoding="utf-8") as f:
    first = json.loads(f.readline())
    print(first)


{'src': 'सोमबारका दिन, स्ट्यानफोर्ड युनिभर्सिटी स्कुल अफ मेडिसिनका वैज्ञानिकहरूले एक नयाँ डायग्नोस्टिक उपकरणको आविष्कारको घोषणा गरे जसले कोषहरूलाई प्रकारका आधारमा क्रमबद्ध गर्न सक्दछः एउटा सानो प्रिन्ट गर्न सकिने चिप जुन मानक ईंकजेट प्रिन्टरहरू प्रयोग गरेर सम्भवतः लगभग एक अमेरिकी सेन्टको लागतमा निर्माण गर्न सकिन्छ।', 'tgt': 'සඳුදා දින, ස්ටැන්ෆර්ඩ් සරසවි වෛද්\u200dය පාසලේ විද්\u200dයාඥයෝ, සෛලයක ආකාරය අනුව එය වර්ග කළ හැකි නව දෝෂ නිර්ණය කිරීමේ මෙවලමක්: එනම්, එකක් ඇමරිකානු ඩොලර් ශතයක පමණ මුදලකින් නිෂ්පාදනය කළ හැකි සාමාන්\u200dය ඉන්ක්ජෙට් මුද්\u200dරණ යන්ත්\u200dරයකින් මුද්\u200dරණය කළ හැකි ඉතා කුඩා චිපයක් සොයාගත් බවට නිවේදනය කළහ.'}


In [13]:
import json

ne_sents = []
si_refs = []

with open("/content/ne_si_dev.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        ne_sents.append(data["src"])
        si_refs.append(data["tgt"])

predicted_si = []

with open("/content/direct_predicted_si.txt", "r", encoding="utf-8") as f:
    for line in f:
        predicted_si.append(line.strip())

In [15]:
from bleurt import score

# Load the BLEURT scorer
checkpoint = "bleurt-base-128"  # Path to the unzipped folder
scorer = score.BleurtScorer(checkpoint)

bleurt_scores = scorer.score(references=si_refs, candidates=predicted_si)

avg = sum(bleurt_scores) / len(bleurt_scores)
print(f"Average BLEURT score: {avg:.4f}")

# Runtime: 6m27s

Average BLEURT score: 0.4267
